# XX Light Source

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt


default_dataset = 'xx_lighttools.txt'
# default_dataset = 'xx_tracepro.txt'

def load_rays(fname=default_dataset):
    data = np.loadtxt(fname, usecols=(0, 1, 2, 3, 4, 5))
    return torch.from_numpy(data).float()

all_rays = load_rays()
assert all_rays.shape == (1000000, 6)

def random_rays(N):
    indices = torch.randint(0, all_rays.shape[0], (N,))
    return all_rays[indices]

def view_rays(rays):
    X, Y, Z, L, M, P = rays.unbind(-1)

    plt.figure()
    plt.scatter(X, Y, s=0.2)
    plt.gca().set_aspect("equal")
    plt.figure()
    plt.scatter(L, M, s=0.2)
    plt.gca().set_aspect("equal")


view_rays(random_rays(1000))


In [ ]:
import roma
import torch

from build123d import *
import numpy as np

import torch
import roma


def normal_to_euler(target_normal):
    "Euler angles in degrees to match target normal"
    
    # vector of rotation
    unit_z = torch.tensor([0., 0., 1.], dtype=torch.float64)
    cross = torch.nn.functional.normalize(torch.linalg.cross(target_normal, unit_z), dim=-1)
    
    # angle of rotation
    angle = -torch.acos(torch.dot(target_normal, unit_z))
    
    return torch.rad2deg(roma.rotvec_to_euler('xyz', angle*cross))


def maketri(position, normal, radius):
    "Make a triangle with the given normal"

    euler = normal_to_euler(normal)
    rot = Rotation(*euler.tolist())

    pos = Pos(*position.tolist())
    
    tri = RegularPolygon(radius=radius, side_count=3)
    #part = extrude(pos * rot * tri, amount=height)
    part = pos * rot * tri
    return part



def produce(rays: torch.Tensor, t: float, radius: float, fname: str):
    P, V = 1000*rays[:, :3], rays[:, 3:]
    P[:, 2] = 1000
    V = torch.nn.functional.normalize(V, dim=-1)

    t = torch.tensor(t, dtype=torch.float64)
    centers = P + t.unsqueeze(-1) * V

    # remove centers that don't fit within the print volume
    fit_volume = torch.logical_and(centers[:, 0].abs() < 500 - radius, centers[:, 1].abs() < 500 - radius)

    P = P[fit_volume]
    V = V[fit_volume]
    centers = centers[fit_volume]


    target = torch.tensor([0., 0., 500.], dtype=torch.float64)
    reflected = torch.nn.functional.normalize(target - centers)

    normals = torch.nn.functional.normalize(reflected - V, dim=-1)

    #parts = [Pos(0, 0, 500)*Sphere(20)]
    #parts += Box(1000, 1000, 1000)
    parts = []

    for C, N in zip(centers, normals):
        part = maketri(C, N, radius)
        parts.append(part)

    xxreflector = Compound(label="xx", children=parts)

    #export_step(xxreflector, f"triangles-{num_rays}.step")
    inmm = scale(xxreflector, by=0.001)
    export_stl(inmm, fname)



def demo():
    num_rays = 33
    # theory radius ~1.51
    radius = 1.1
    rays = random_rays(num_rays)
    view_rays(rays)

    xxreflector = produce(rays, 900, radius, f"triangles-{num_rays}.stl")

    if num_rays <= 100:
        display(xxreflector)


def chunks():
    size = 50
    nchunks = 1
    starti = 0
    radius = 2.0
    print(f"Making {nchunks} chunks of {size} triangles each. Total = {nchunks*size} ({nchunks*size / 1e6:.2%})")

    for i in range(nchunks):
        start, end = starti + i*size, (starti + i+1)*size
        rays = all_rays[start:end, :]

        fname = f"chunks/chunk-{size}-{i}.stl"
        print(f"{i+1} / {nchunks}", end="\r")
        produce(rays, t=900, radius=radius, fname=fname)
    print()

chunks()